# 빅분기Part3-모델링
p.278

방식1

In [ ]:
### 시간 측정, 데이터 로드, 종속-독립변수 분리
start_time= time.time()
import pandas as pd
data=pd.read_csv('bigData/mtcars.csv')
# print(data.info(), '-----'))
# print(data.head(), '-----'))
Y= data[['mpg']]
X= data.drop(columns=['mpg','Unnamed: 0'])
X.head()
### null 찾고 채우기, 이상한 값 찾고 바꾸기, 이상한 타입 바꾸기, 인코딩
# print(X.loc[X.isnull().sum(1) != 0], '-----'))
X['cyl'].fillna(X.describe().loc['mean','cyl'], inplace=True)
X['qsec'].fillna(X.describe().loc['mean','cyl'], inplace=True)
# print(X, '-----'))
# X.loc[:,X.dtypes==object].iloc[:,0].unique()
# print(X.loc[:,X.dtypes==object].iloc[:,1].unique(), '-----')) #'gear'
# print(X.info(), '-----')) 
X['gear']= X['gear'].replace('*3','3').replace('*5','5').astype('int64') # min 이상치에 'gear' 도 포함된다
# help(pd.get_dummies)
# print(pd.get_dummies(X, drop_first=True), '-----'))
# print(X['am'].unique(), '-----'))
# print(X, '-----'))
X= pd.get_dummies(X, drop_first=True)
# print(X, '-----')) # am_auto 뿐만 아니라, 기존 am 컬럼도 자동으로 지워진다.

# print(X.info(), '-----'))
### 이상치 교체(meanStd or IQR)
## describe -> mean +- 1.5*std , Q3 + 1.5*IQR 및 Q1 - 1.5*IQR -> def 컬럼 별. maxOver minOver. 찾는 것부터
# 같은 컬럼을 2번 돌리면 안 되기 때문에 True 라고 전부 loop 돌리면 곤란하다. 
# int64 를 float로? 눈으로 데이터보며 이상치 제거할지 말지 결정하자.
# print(X.loc[:,X.dtypes=='int64'].nunique(), '-----')) # hp22, vs2, gear3, carb6 종
# print(X.loc[:,X.dtypes=='int64'], '-----')) # vs 01, gear 345, carb 123468 
# 컬럼 조건만 알아본 것
def outlierCheck(data):
    desc= data.describe()
    max1= desc.loc['max']
    min1= desc.loc['min']
    
    mean= desc.loc['mean']
    std= desc.loc['std']
    minBms= mean - 1.5*std
    maxBms= mean + 1.5*std
    minOverMs= min1 < minBms#조건을 잘못 걸었다. Boundary 보다 더 작은 게 이상치. minOverMs= min1 > minBms
    maxOverMs= max1 > maxBms #Boundary 보다 더 크면 이상치. maxOverMs= max1 < maxBms
#     print(minOverMs, '-----')) # drat, wt 만
#     print(maxOverMs, '-----')) # vs, am_maunal 빼고 다 => disp, drat 만. gear 제외
    # 여기부터 의심했어야 했는데. vs 01인데 왜 이상치로 잡나에 대해.
    
    Q1= desc.loc['25%']
    Q3= desc.loc['75%']
    IQR= Q3-Q1
    minBiqr= Q1 - 1.5*IQR
    maxBiqr= Q3 + 1.5*IQR
    minOverIQR= min1 < minBiqr
    maxOverIQR= max1 > maxBiqr
#     print(minOverIQR, '-----')) # qsec 만
#     print(maxOverIQR, '-----')) # cyl, hp, wt, qsec, carb => carb 유지. 기화기는 엔진 부품. 8개면 좀 이상하긴하다
    
    #     print(minBms, maxBms, minBiqr, maxBiqr, '-----'))
    return minBms, maxBms, minBiqr, maxBiqr

# outlierCheck(X)

# 뭘 바꿀지 이미 정했으므로 행 조건만 알면 된다.
def outlierReplace(data):
    minBms, maxBms, minBiqr, maxBiqr= outlierCheck(data)
    #행 조건
    # 각 열 내의 값이 Boundary Over 인지
    #열 조건
    minMScols= ['drat','wt']
    maxMScols= ['disp','drat']
    minIQRcols= ['qsec']
    maxIQRcols= ['cyl', 'hp', 'wt', 'qsec', 'carb']
    
    #     print(type(minBms), '-----'))
    #     print(minBms[column], '-----'))
    for col in minMScols:
#         print(f"BeforeMinMS: {data.loc[ data[col]<minBms[col],col]}, minBms: {minBms[col]:.7}") 
        data.loc[ data[col]<minBms[col], col]= minBms[col]
    for col in maxMScols:
#         print(f"BeforeMaxMS: {data.loc[ data[col]>maxBms[col],col]}, maxBms: {maxBms[col]:.7}")
        data.loc[ data[col]>maxBms[col], col]= maxBms[col]
    for col in minIQRcols:
#         print(f"BeforeMinIQR: {data.loc[ data[col]<minBiqr[col],col]}, minBiqr: {minBiqr[col]:.7}")
        data.loc[ data[col]<minBiqr[col], col]= minBiqr[col]
    for col in maxIQRcols:
#         print(f"BeforeMaxIQR: {data.loc[ data[col]>maxBiqr[col],col]}, maxBiqr: {maxBiqr[col]:.7}")
        data.loc[ data[col]>maxBiqr[col], col]= maxBiqr[col]
    return data

X= outlierReplace(X) # inplace 될 때도 있고, 안 될 때도 있어서 확실히 해두기.
# print(X, '-----'))

### 범위 맞추기
# from sklearn import preprocessing
# help(preprocessing)
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder
# print(X.describe(), '-----'))
def rangeScaler(scalers, columnDataFrame): # 이 인자 외워야. X[[ ]]
    if scalers=='MinMax':
        scaler= MinMaxScaler()
    elif scalers=='Standard':
        scaler= StandardScaler()
    elif scalers=='Robust':
        scaler= RobustScaler()
    #     help(scaler.fit_transform)
    scaledDf= pd.DataFrame(scaler.fit_transform(columnDataFrame)) # 이 형태 외워야.
    return scaledDf

# print(X[['qsec']], '-----'))
ret2= rangeScaler('MinMax', X[['qsec']]) # min 0, max 1
ret2= rangeScaler('Standard', X[['qsec']]) # mean 0 근접, std 1 근접
ret2= rangeScaler('Robust', X[['qsec']]) # mid 0, IQR= Q3-Q1= 1
# print(ret2.describe(), '-----'))
# ret2.describe().loc['75%']-ret2.describe().loc['25%']
# print(ret2, '-----'))

## 범주형은 안 건드리고 싶다.
# help(X['am_manual'].astype) #Convert to categorical type
X['am_manual']= X['am_manual'].astype('category')
X['vs']= X['vs'].astype('category')
# print(X.info(), '-----'))


## LabelEncoder 연습
face= ['happy','sad','soso']
# from sklearn import preprocessing
# help(preprocessing)
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
# print(face, encoder.fit_transform(face), '-----'))
